In [15]:
import os
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from tqdm.notebook import tqdm

chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--incognito')

with open('naver_total.json', 'r', encoding='utf-8') as file:
    data = json.load(file)
    extracted_id = [item.get('id') for item in data if 'id' in item]

results = {}
web_error_ids = []
elements_to_click_error_ids = []
elements_to_append_error_ids = []
extracted_id = extracted_id[:]

for id in tqdm(extracted_id, desc='Processing batches'):
    hospital_info_elements = []

    driver = None

    try:
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(f'https://m.place.naver.com/place/{id}/home')
    except Exception as e:
        web_error_ids.append(id)
    else:
        try:
            driver.implicitly_wait(0.5)
            element_with_class = driver.find_elements(By.CSS_SELECTOR, 'span._UCia svg')[1].click()
        except Exception as e:
            elements_to_click_error_ids.append(id)
        try:
            driver.implicitly_wait(0.5)
            element_with_class = driver.find_element(By.CSS_SELECTOR, 'span.rvCSr svg').click()
        except Exception as e:
            elements_to_click_error_ids.append(id)
        elements_to_append = [
            'a.gKP9i.RMgN0 > div',
            'ul.Ni7dC > li',
            'div.vV_z_ > a.xHaT3 > span.zPfVt'
        ]
        for element_selector in elements_to_append:
            for element in driver.find_elements(By.CSS_SELECTOR, element_selector):
                try:
                    driver.implicitly_wait(0.5)
                    hospital_info_elements.append(element.text)
                except Exception as e:
                    elements_to_append_error_ids.append(id)

    finally:
        if driver:
            driver.quit()

    if hospital_info_elements:
        results[id] = hospital_info_elements

print(f"A total of {len(web_error_ids)} web errors occurred for the following IDs")
print(f"A total of {len(elements_to_click_error_ids)} errors clicking elements occurred for the following IDs")
print(f"A total of {len(elements_to_append_error_ids)} errors appending elements occurred for the following IDs")

for id, info in results.items():
    if not os.path.exists('hospital_info'):
        os.makedirs('hospital_info')
    with open(f'hospital_info/{id}_info.json', 'a+', encoding='utf-8') as file:
        json.dump({id: info}, file, indent=4, ensure_ascii=False)

Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]

A total of 0 web errors occurred for the following IDs
A total of 0 errors clicking elements occurred for the following IDs
A total of 0 errors appending elements occurred for the following IDs
